In [2]:
!pip install newsapi-python
!pip install pandas numpy requests

# 1. Configuración de API Keys

In [44]:
import pandas as pd
import numpy as np
import requests
from datetime import datetime, timedelta
from newsapi import NewsApiClient


# API Key para NewsAPI (regístrate en https://newsapi.org/)
NEWS_API_KEY = ""  # <-- Reemplaza aquí

# API Key para OpenWeatherMap (regístrate en https://openweathermap.org/)
OPENWEATHER_API_KEY = ""  # <-- Reemplaza aquí

# API Key para Tipos de cambio (regístrate en https://www.exchangerate-api.com/)
FX_API_KEY = ""

# Nota: La API del BCRP no requiere API key para consultar las series.

# 2. Extracción de Datos Externos

In [10]:
def fetch_news_data():
    # Inicializa el cliente de NewsAPI con la API Key
    newsapi = NewsApiClient(api_key=NEWS_API_KEY)

    # Define una búsqueda relacionada a temas económicos y de negocios
    query = "negocios OR economía OR inversión OR empresa OR finanzas"

    # Lista de dominios de medios peruanos (separados por comas)
    dominios = "elcomercio.pe,larepublica.pe,gestion.pe,peru21.pe,elpopular.pe"

    # Consulta a la API usando la lista de dominios específicos
    response = newsapi.get_everything(
        q=query,
        language="es",
        domains=dominios
    )
    articles = response.get("articles", [])

    # Extrae la información relevante de cada artículo y la almacena en una lista
    news_list = []
    for art in articles:
        news_list.append({
            "titulo": art.get("title"),
            "descripcion": art.get("description"),
            "fecha": art.get("publishedAt"),
            "fuente": art.get("source", {}).get("name")
        })

    # Convierte la lista en un DataFrame (asegúrate de importar pandas como pd)
    df = pd.DataFrame(news_list)

    return df

In [13]:
## b) Temperatura en Perú por Ciudad usando OpenWeatherMap
def fetch_temperature_data(city):
    url = "http://api.openweathermap.org/data/2.5/weather"
    params = {
        "q": f"{city},pe",  # Ciudad en Perú
        "appid": OPENWEATHER_API_KEY,
        "units": "metric",  # Temperatura en Celsius
        "lang": "es"
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        data = response.json()
        temp_data = {
            "ciudad": city,
            "temperatura": data.get("main", {}).get("temp"),
            "humedad": data.get("main", {}).get("humidity"),
            "fecha": datetime.utcfromtimestamp(data.get("dt")).strftime("%Y-%m-%d %H:%M:%S")
        }
        return temp_data
    else:
        print(f"Error al obtener datos para {city}: {response.status_code}")
        return None

In [56]:
def get_pen_to_usd_rate():
    """
    Obtiene el tipo de cambio de Soles peruanos (PEN) a Dólar estadounidense (USD)
    usando la API de ExchangeRate-API.

    La URL de consulta define PEN como la moneda base, por lo que la API retorna el valor de USD
    en 'conversion_rates' como cuántos USD equivale 1 PEN.
    Para obtener el tipo de cambio en términos de cuántos soles se requieren para 1 dólar (ej. 3.70),
    se invierte ese valor.

    Retorna:
      - rate_inverted: el valor de 1 USD en PEN.
      - update_time: la fecha/hora de la última actualización (en formato UTC).
    """
    url = f"https://v6.exchangerate-api.com/v6/{FX_API_KEY}/latest/PEN"

    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        data = response.json()

        # Verifica que el resultado de la API sea "success"
        if data.get("result") != "success":
            print("Error en la respuesta de la API:")
            print(data)
            return None, None

        conversion_rates = data.get("conversion_rates", {})
        rate_usd = conversion_rates.get("USD")
        update_time = data.get("time_last_update_utc")

        if rate_usd is None or rate_usd == 0:
            print("No se obtuvo una tasa válida.")
            return None, None

        # Invierte la tasa para obtener cuántos PEN se requieren por 1 USD.
        rate_pen_per_usd = 1 / rate_usd
        return rate_pen_per_usd, update_time

    except requests.RequestException as e:
        print(f"Error al obtener los datos: {e}")
        return None, None

In [65]:
## Extraer noticias
df_news = fetch_news_data()

## Extraer Temperaturas
# Lista de las capitales de los departamentos de Perú
capitales = [
    "Chachapoyas", "Huaraz", "Abancay", "Arequipa", "Ayacucho", "Cajamarca", "Callao",
    "Cusco", "Huancavelica", "Huánuco", "Ica", "Huancayo", "Trujillo", "Chiclayo",
    "Lima", "Iquitos", "Puerto Maldonado", "Moquegua", "Cerro de Pasco", "Piura",
    "Puno", "Moyobamba", "Tacna", "Tumbes", "Pucallpa"
]

temp_data_list = [fetch_temperature_data(capital) for capital in capitales]
temp_data_list = [d for d in temp_data_list if d is not None]
df_temp = pd.DataFrame(temp_data_list)

## Extraer tipo de cambio
rate, update_time = get_pen_to_usd_rate()

df_exchange = pd.DataFrame({"fecha": [update_time], "tipo_cambio": [rate]})

In [74]:
df_exchange.columns

Index(['fecha', 'tipo_cambio'], dtype='object')

# 3. Aplicación de Reglas de Calidad de Datos (Reglas Básicas)

In [69]:
def check_news_technical(df_news):
    issues = []
    # Completitud: se requiere que existan todas las columnas esenciales.
    required_columns = ['titulo', 'descripcion', 'fecha', 'fuente']
    for col in required_columns:
        if col not in df_news.columns:
            issues.append(f"Falta la columna '{col}'.")

    # Comprobación de valores nulos
    for col in required_columns:
        if col in df_news.columns and df_news[col].isnull().any():
            issues.append(f"Existen valores nulos en la columna '{col}'.")

    # Duplicados: suponemos que 'titulo' y 'fecha' deben ser únicos
    dup_count = df_news.duplicated(subset=['titulo', 'fecha']).sum()
    if dup_count > 0:
        issues.append(f"Se encontraron {dup_count} filas duplicadas (basadas en 'titulo' y 'fecha').")

    return issues

def check_temp_technical(df_temp):
    issues = []
    required_columns = ['ciudad', 'temperatura', 'humedad', 'fecha']
    for col in required_columns:
        if col not in df_temp.columns:
            issues.append(f"Falta la columna '{col}'.")

    for col in required_columns:
        if col in df_temp.columns and df_temp[col].isnull().any():
            issues.append(f"Existen valores nulos en la columna '{col}'.")

    # Duplicados: por ejemplo, una misma medición por región y fecha
    dup_count = df_temp.duplicated(subset=['ciudad', 'fecha']).sum()
    if dup_count > 0:
        issues.append(f"Se encontraron {dup_count} filas duplicadas (basadas en 'ciudad' y 'fecha').")

    return issues

def check_exchange_technical(df_exchange):
    issues = []
    required_columns = ['fecha', 'tipo_cambio']
    for col in required_columns:
        if col not in df_exchange.columns:
            issues.append(f"Falta la columna '{col}'.")

    for col in required_columns:
        if col in df_exchange.columns and df_exchange[col].isnull().any():
            issues.append(f"Existen valores nulos en la columna '{col}'.")

    # Verificar duplicados en la fecha (suponemos que cada día debe aparecer solo una vez)
    dup_count = df_exchange.duplicated(subset=['fecha']).sum()
    if dup_count > 0:
        issues.append(f"Se encontraron {dup_count} filas duplicadas en la columna 'fecha'.")

    return issues

# 4. Aplicación de Reglas de Calidad de Datos (Reglas de Negocio)

In [70]:
def check_news_business(df_news):
    issues = []
    # Palabras clave que deben aparecer en noticias relevantes
    business_keywords = ['negocio', 'economía', 'inversión', 'empresa', 'finanza']

    # Se usa la función str.contains para verificar la presencia de alguna palabra clave (case insensitive)
    mask = (df_news['titulo'].str.lower().str.contains('|'.join(business_keywords), na=False) |
            df_news['descripcion'].str.lower().str.contains('|'.join(business_keywords), na=False))

    # Consideramos como problema aquellos artículos que no cumplan con la condición
    invalid_news = df_news[~mask]
    if not invalid_news.empty:
        issues.append(f"{len(invalid_news)} artículos no parecen estar relacionados a negocios/inversiones.")
    return issues

def check_temp_business(df_temp):
    issues = []
    # Definir un rango realista para las temperaturas en Perú.
    # Este rango puede ajustarse, por ejemplo, entre -10 y 40 grados Celsius.
    invalid_temps = df_temp[(df_temp['temperatura'] < -10) | (df_temp['temperatura'] > 40)]
    if not invalid_temps.empty:
        issues.append("Se encontraron temperaturas fuera del rango esperado (−10 a 40 °C): " +
                      str(invalid_temps[['ciudad', 'temperatura', 'fecha']].to_dict(orient='records')))
    return issues

def check_exchange_business(df_exchange):
    issues = []
    # Regla de negocio: Se espera que el tipo de cambio (1 USD en PEN) esté entre 3.60 y 3.80.
    outliers = df_exchange[(df_exchange['tipo_cambio'] < 3.60) | (df_exchange['tipo_cambio'] > 3.80)]
    if not outliers.empty:
        issues.append("Existen valores del tipo de cambio fuera del rango esperado (3.60 - 3.80 PEN por USD): " +
                      str(outliers[['fecha', 'tipo_cambio']].to_dict(orient='records')))

    # Otra posible regla: variaciones diarias excesivas, por ejemplo, cambios mayores al 5% respecto al día anterior.
    df_exchange_sorted = df_exchange.sort_values("fecha")
    df_exchange_sorted['prev_rate'] = df_exchange_sorted['tipo_cambio'].shift(1)
    df_exchange_sorted['variation'] = ((df_exchange_sorted['tipo_cambio'] - df_exchange_sorted['prev_rate']) /
                                       df_exchange_sorted['prev_rate']).abs() * 100
    abrupt_changes = df_exchange_sorted[df_exchange_sorted['variation'] > 5]
    if not abrupt_changes.empty:
        issues.append("Se detectaron variaciones diarias superiores al 5%: " +
                      str(abrupt_changes[['fecha', 'tipo_cambio', 'variation']].to_dict(orient='records')))
    return issues

# 4. Resumen de KPIs de Calidad de Datos

In [75]:
def run_data_quality_checks(df_news, df_temp, df_exchange):
    report = {
        'Noticias': {
            'reglas_basicas': check_news_technical(df_news),
            'reglas_de_negocio': check_news_business(df_news)
        },
        'Temperatura': {
            'reglas_basicas': check_temp_technical(df_temp),
            'reglas_de_negocio': check_temp_business(df_temp)
        },
        'Tipo de cambio': {
            'reglas_basicas': check_exchange_technical(df_exchange),
            'reglas_de_negocio': check_exchange_business(df_exchange)
        }
    }
    return report

# ----- Supongamos que ya tienes tus dataframes df_news, df_temp y df_exchange -----
# Por ejemplo, pueden haber sido cargados con:
# df_news = pd.read_csv("news_data.csv")
# df_temp = pd.read_csv("temp_data.csv")
# df_exchange = pd.read_csv("exchange_data.csv")

# Aquí se simula llamando a la función de validación (recuerda adaptar según tus datos reales):
report = run_data_quality_checks(df_news, df_temp, df_exchange)

# ----- Generar y mostrar el reporte en el formato solicitado -----
print("\nReporte de Calidad de Datos\n")
print("1. Reglas básicas")
# Noticias
if not report['Noticias']['reglas_basicas']:
    print("   - Noticias: Cumple con las reglas técnicas.")
else:
    print("   - Noticias: No cumple con las reglas técnicas:")
    for issue in report['Noticias']['reglas_basicas']:
        print("       *", issue)
# Temperatura
if not report['Temperatura']['reglas_basicas']:
    print("   - Temperatura: Cumple con las reglas técnicas.")
else:
    print("   - Temperatura: No cumple con las reglas técnicas:")
    for issue in report['Temperatura']['reglas_basicas']:
        print("       *", issue)
# Tipo de cambio
if not report['Tipo de cambio']['reglas_basicas']:
    print("   - Tipo de cambio: Cumple con las reglas técnicas.")
else:
    print("   - Tipo de cambio: No cumple con las reglas técnicas:")
    for issue in report['Tipo de cambio']['reglas_basicas']:
        print("       *", issue)

print("\n2. Reglas de negocio")
# Noticias
if not report['Noticias']['reglas_de_negocio']:
    print("   - Noticias: Cumple con las reglas de negocio.")
else:
    print("   - Noticias: " + str(report['Noticias']['reglas_de_negocio'][0]))
# Temperatura
if not report['Temperatura']['reglas_de_negocio']:
    print("   - Temperatura: Cumple con las reglas de negocio.")
else:
    print("   - Temperatura: " + str(report['Temperatura']['reglas_de_negocio'][0]))
# Tipo de cambio
if not report['Tipo de cambio']['reglas_de_negocio']:
    print("   - Tipo de cambio: Cumple con las reglas de negocio.")
else:
    issues_tc = report['Tipo de cambio']['reglas_de_negocio']
    print("   - Tipo de cambio:")
    for issue in issues_tc:
        print("       *", issue)

# Para mayor detalle, puedes optar por imprimir el diccionario completo de reporte:
#print("\nReporte completo:")
#pprint.pprint(report)


Reporte de Calidad de Datos

1. Reglas básicas
   - Noticias: Cumple con las reglas técnicas.
   - Temperatura: Cumple con las reglas técnicas.
   - Tipo de cambio: Cumple con las reglas técnicas.

2. Reglas de negocio
   - Noticias: 34 artículos no parecen estar relacionados a negocios/inversiones.
   - Temperatura: Cumple con las reglas de negocio.
   - Tipo de cambio: Cumple con las reglas de negocio.
